In [1]:
#1. 사용환경 준비
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ") # API 키 입력

OpenAI API key 입력:  ········


In [2]:
#2. 모델 로드하기 
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
#3. 문서 로드하기
from langchain.document_loaders import CSVLoader

# csv 파일 로드.
loader = CSVLoader("Cultural_asset.csv",encoding='UTF8')
loader2 = CSVLoader("Travel_spot.csv",encoding='UTF8')

# 페이지 별 문서 로드
Treasures = loader.load()
Travel = loader2.load()
print(Travel[:1])

[Document(metadata={'source': 'Travel_spot.csv', 'row': 0}, page_content='title: 구인사(단양)\naddress: 충청북도 단양군 영춘면 구인사길 73\ntelNo: ')]


In [4]:
#4. 문서 청크로 나누기(CharacterTextSplitter)
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document



# 텍스트 청크 분할기 설정 (문단 기준 분할)
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(Treasures) # 문서를 청크로 분할
splits2 = text_splitter.split_documents(Travel)
print(splits2[1]) # 상위 1개만 출력

page_content='title: 원대리 자작나무 숲 (속삭이는 자작나무 숲)
address: 강원특별자치도 인제군 인제읍 자작나무숲길 760
telNo: 자작나무숲 안내소 033-463-0044' metadata={'source': 'Travel_spot.csv', 'row': 1}


In [5]:
#5 벡터 임베딩 생성
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [6]:
#6. 벡터 저장소 생성
import faiss
from langchain_community.vectorstores import FAISS

# 문서에서 벡터 저장소 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

In [7]:
#(수정)Travel 데이터를 기반으로 벡터 저장소 생성
travel_vectorstore = FAISS.from_documents(documents=splits2, embedding=embeddings)

In [8]:
#7. FAISS를 Retriever로 변환
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [9]:
#(수정)Travel 데이터를 위한 Retriever 생성
travel_retriever = travel_vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [29]:
#8. 프롬프트 템플릿을 정의하라
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context. Do not include any external information. Respond in Korean only."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [34]:
# (수정) 띄어쓰기 문제 개선 코드 써봤지만 실패
import re

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template

    def normalize_spacing(self, text):
        # 모든 연속된 공백을 하나의 공백으로 줄임
        return re.sub(r'\s+', ' ', text).strip()

    def invoke(self, inputs):
        if isinstance(inputs, list):
            # 페이지 내용을 정리하여 하나의 텍스트로 연결, 띄어쓰기를 정규화
            context_text = "\n".join([self.normalize_spacing(doc.page_content.strip()) for doc in inputs])
        else:
            context_text = self.normalize_spacing(inputs.strip())

        # 프롬프트에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "").strip() if isinstance(inputs, dict) else inputs.strip()
        )
        return formatted_prompt

In [33]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [92]:
from langchain.document_loaders import CSVLoader

# Cultural_asset.csv 파일을 로드하고 데이터를 가져옵니다.
loader = CSVLoader("Cultural_asset.csv", encoding='UTF8')

# 데이터를 로드하여 Treasures 변수에 저장
Treasures = loader.load()

# 데이터를 제대로 로드했는지 확인
print(Treasures[:1])  # 첫 번째 데이터 출력


[Document(metadata={'source': 'Cultural_asset.csv', 'row': 0}, page_content='number: 1호\ndesignation: 서울 숭례문\nlocation: 서울 중구 세종대로 40 (남대문로4가)')]


In [136]:
from fuzzywuzzy import process, fuzz
import pandas as pd
import re

# CSV 파일에서 여행지 데이터 읽기
def load_travel_data(file_path):
    try:
        df = pd.read_csv(file_path)
        return df[['title', 'address']].values.tolist()  # 'title'과 'address'만 추출
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return []
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        return []

# 문화재 정보가 저장된 CSV 파일을 읽어오는 함수
def load_treasure_data(file_path):
    try:
        df = pd.read_csv(file_path)
        return df[['designation', 'location']].values.tolist()  # 'designation'과 'location'만 추출
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return []
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        return []

# 문화재명에서 괄호와 영어 설명 제거하는 함수
def clean_designation(designation):
    # 괄호와 괄호 안의 내용 제거
    cleaned = re.sub(r'\(.*\)', '', designation)
    # 양 옆의 공백 제거 후 반환
    return cleaned.strip()

# 여행지 위치 비교 클래스
class LocationMatcher:
    def __init__(self, travel_data):
        self.travel_data = travel_data

    def match_location(self, treasure_location):
        # 여행지의 주소만 가져오기
        travel_addresses = [str(spot[1]) for spot in self.travel_data]

        # 문화재 위치와 가장 유사한 여행지 주소 찾기
        best_matches = process.extract(treasure_location, travel_addresses, scorer=fuzz.ratio, limit=5)

        # 유사도가 높은 상위 3개 제외 부분 제거
        best_match_data = []
        for match in best_matches:
            match_data = [spot for spot in self.travel_data if str(spot[1]) == match[0]]
            if match_data:
                best_match_data.append(match_data[0])

        return best_match_data

# CSV 파일에서 데이터 읽기
travel_spots = load_travel_data('travel_spot.csv')  # 여행지 데이터
treasure_data = load_treasure_data('Cultural_asset.csv')  # 문화재 데이터

# 위치 비교 인스턴스 생성
location_matcher = LocationMatcher(travel_spots)

In [ ]:
# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    if query == "종료":  # 종료 입력 시 챗봇 종료
        print("챗봇을 종료합니다.")
        break

    # 1. 사용자 질문에 해당하는 문화재명 찾기
    treasure_name = query.strip()

    # 2. 문화재 정보 처리 (treasure_name에 해당하는 위치 가져오기)
    treasure_location = next(
        (location for designation, location in treasure_data if clean_designation(designation) == treasure_name),
        None
    )

    if treasure_location:
        print(f"\n■문화재 위치: {treasure_location}\n")

        # 여행지 위치와 비교하여 추천
        best_match_data = location_matcher.match_location(treasure_location)

        if best_match_data:
            print(f"■추천 여행지:")
            for title, address in best_match_data:
                print(f"   - {title} \n     주소({address})")
        else:
            print("추천된 여행지가 유효하지 않습니다.")
    else:
        print(f"{treasure_name}에 대한 문화재 정보가 없습니다.")

질문을 입력하세요 :  김완초상



■문화재 위치: 전남 영암군 서호면 화송리 159

■추천 여행지:
   - 왕벚165 
     주소(전라남도 영암군 서호면 왕인로 165)
   - 구고사 및 김완장군부조묘 
     주소(전라남도 영암군 서호면 화소길 20)
   - F1 오토캠핑장 
     주소(전라남도 영암군 삼호읍 삼포리 1894)
   - 죽정서원 
     주소(전라남도 영암군 군서면 죽정서원길 19)
   - 영암도기박물관 
     주소(전라남도 영암군 군서면 서호정길 5)


질문을 입력하세요 :  김완 초 상


김완 초 상에 대한 문화재 정보가 없습니다.


질문을 입력하세요 :  김완 초상


김완 초상에 대한 문화재 정보가 없습니다.
